In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import src.gen_source as gen_source
import src.two_tone_lib as tt

import src.tcp_async as tcp_async
import src.tcp_sync as tcp_sync

from live_analyse_py import live_analyse_py

In [ ]:
try:
    __IPYTHON__
    reload(tcp_async)
    reload(tcp_sync)
    reload(gen_source)
    reload(tt)
except:
    pass

In [ ]:
sync = tcp_sync.UhdSyncMsg(packet_size=4*8192,
                           packet_type="".join(["f"]*8192))
async = tcp_async.UhdAsyncMsg()

In [ ]:
top = live_analyse_py()

In [ ]:
top.start()

In [ ]:
sync.has_msg()

In [ ]:
def measure(param):
    x2 = param[0]
    x3 = param[1]
    x4 = param[2]
    repeat = True
    while repeat:
        tt.gen_two_tone(debug = True, predist=tt.predist_poly, par=(x2, x3, x4))
        sync.has_msg()
        np.array(sync.get_msgs(2))
        msgs = np.array(sync.get_msgs(5))
        msgs = [np.fft.fftshift(msg) for msg in msgs]
        
        if async.has_msg():
            continue
            
        a = np.array(msgs)
        mean_msg = a.mean(axis = 0)
        suffix = "x_2_%.3f_x_3_%.3f_x_4_%.3f" % (x2, x3, x4)
        sig_to_noise = tt.analyse_power_spec(mean_msg, debug=True, debug_path="/tmp/out", suffix=suffix)
        print(sig_to_noise, x2, x3, x4)
        repeat = False

In [ ]:
top.set_txgain(85)

params = []
for x2 in np.linspace(-0.1, 0.1, num = 11):
    for x3 in np.linspace(-0.1, 0.1, num = 11):
        for x4 in np.linspace(-0.1, 0.1, num = 11):
            params.append((x2, x3, x4))
            
t_start = time.time()
for idx, param in enumerate(params):
    measure(param)
    time_per_element = (time.time() - t_start) / (idx + 1)
    print ("Time per Element " + str(time_per_element) +
           ", total: " + str(time_per_element * len(params)),
           ", left: " + str(time_per_element * (len(params) - 1 - idx))
          )

In [ ]:
sync.stop()
async.stop()
top.stop()
top.wait()